In [4]:
import numpy as np
import librosa as lp
import subprocess
import os
from IPython.core.debugger import set_trace
from sklearn.model_selection import train_test_split
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from scikits.talkbox import lpc

print(1)
# This function is to convert mp3 to wav currently in disuse
def convert_audio_to_wav(filename):
    subprocess.call(['ffmpeg', '-i', filename,
                   str(filename.split(".mp3")[0])+".wav"])
    
#Function to convert mp3 audio to lpc shapes    
def convert_to_lpc(filename,number_of_coefficients):
    wave, sr = lp.load(filename, mono=True, sr=None)
    lpc_signal=lpc(wave,number_of_coefficients)
    processed_lpc_signal=[[0.00 for x in range(number_of_coefficients+1)] for y in range(len(lpc_signal)) ]
    for i in range(len(lpc_signal)):
        for j in range(len(lpc_signal[i])):
            processed_lpc_signal[i][j]=lpc_signal[i][j]
    return processed_lpc_signal
                    


def create_training_data(pathname_training,number_of_coefficients):
    #set_trace()
    lpc_array=[]
    subdirs=[x[1] for x in os.walk(pathname_training)][0]
    for subdir in subdirs:
        subdir_path=pathname_training+"\\" + subdir
        files2=[x[2] for x in os.walk(subdir_path)][0]
        for file_name in files2:
            file_path=subdir_path+"\\"+file_name
            lpc_of_file=convert_to_lpc(file_path,number_of_coefficients)
            lpc_array.append(lpc_of_file)
        np.save(os.getcwd()+"\\lpc_files\\"+subdir,lpc_array)
        lpc_array=[]

 

def get_lpc_from_file(location_of_lpc):
    lpc_files=os.listdir(location_of_lpc)
    return lpc_files

#get_lpc_from_file( os.getcwd()+"\\lpc_files")

def get_train_test(ratio_of_split, random_state=42):
    # Get available labels
    lpc_labels= get_lpc_from_file( os.getcwd()+"\\lpc_files")

    # Getting first arrays
    X = np.load(os.getcwd()+"\\lpc_files\\"+lpc_labels[0])
    y = np.zeros(X.shape[0])
    

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(lpc_labels[1:]):
        x = np.load(os.getcwd()+"\\lpc_files\\"+label)
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    return train_test_split(X, y, test_size= (1 - ratio_of_split), random_state=random_state, shuffle=True)

def create_model(number_of_coefficients):
    create_training_data("C:\Users\Ashok\Documents\GitHub\SpeakerRecognition\Training",number_of_coefficients) 
    X_train, X_test, y_train, y_test = get_train_test(0.6)
    X_train = X_train.reshape(X_train.shape[0], 3,number_of_coefficients+1, 1)
    X_test = X_test.reshape(X_test.shape[0], 3,number_of_coefficients+1, 1)
    y_train_hot = to_categorical(y_train)
    y_test_hot = to_categorical(y_test)
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(3,number_of_coefficients+1, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    model.fit(X_train, y_train_hot, batch_size=100, epochs=50, verbose=1, validation_data=(X_test, y_test_hot))
    

create_model(20)

1
Train on 540 samples, validate on 360 samples
Epoch 1/50
540/540 [==============================] - 0s 433us/step - loss: 1.8987 - acc: 0.2889 - val_loss: 1.8220 - val_acc: 0.3139
Epoch 2/50
540/540 [==============================] - 0s 463us/step - loss: 1.8011 - acc: 0.3352 - val_loss: 1.7327 - val_acc: 0.3139
Epoch 3/50
540/540 [==============================] - 0s 463us/step - loss: 1.7104 - acc: 0.3593 - val_loss: 1.6219 - val_acc: 0.3139: 0s - loss: 1.7155 - acc: 0.356
Epoch 4/50
540/540 [==============================] - 0s 493us/step - loss: 1.6026 - acc: 0.3907 - val_loss: 1.4644 - val_acc: 0.4667
Epoch 5/50
540/540 [==============================] - 0s 550us/step - loss: 1.4600 - acc: 0.5148 - val_loss: 1.3527 - val_acc: 0.3778
Epoch 6/50
540/540 [==============================] - 0s 463us/step - loss: 1.3299 - acc: 0.5741 - val_loss: 1.1999 - val_acc: 0.4806
Epoch 7/50
540/540 [==============================] - 0s 433us/step - loss: 1.1632 - acc: 0.6463 - val_loss: 0.9842 